<a href="http://www.calstatela.edu/centers/hipic"><img align="left" src="https://avatars2.githubusercontent.com/u/4156894?v=3&s=100"><image/>
</a>
<img align="right" alt="California State University, Los Angeles" src="http://www.calstatela.edu/sites/default/files/groups/California%20State%20University%2C%20Los%20Angeles/master_logo_full_color_horizontal_centered.svg" style="width: 360px;"/>

#    CIS5560 Term Project Tutorial

------
#### Authors: Farnood Rahbar Far, Samyuktha Muralidharan, Sanjana Boddireddy, Savita Yadav

#### Instructor: [Jongwook Woo](https://www.linkedin.com/in/jongwook-woo-7081a85)

#### Date: 05/10/2021

##Import Spark SQL and Spark ML Libraries
Import all the Spark SQL and ML libraries as mentioned below. This is neccessary to access the functions available in those libraries.

In [0]:
# Import Spark SQL and Spark ML libraries
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler,StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, TrainValidationSplit
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import RandomForestRegressor

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

## To run the code in PySpark CLI
Set the following to True:
```
PYSPARK_CLI = True
```
Generate .py(Python) file from Databricks: File > Export > Source File
```
Run it at the Hadoop/Spark cluster:
$ spark-submit Random Forest Regression.py
```

In [0]:
PYSPARK_CLI = False
if PYSPARK_CLI:
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)

##Read the csv file from DBFS (Databricks File System)
Locate the data file, mention its type and read the file as a pyspark dataframe

The link to the sampled file : https://www.kaggle.com/samyukthamurali/airbnb-ratings-dataset?select=airbnb_sample.csv. You can download the sample file from here and upload it in DBFS.

In [0]:
# File location and type
file_location = "/FileStore/tables/airbnb_sample.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
# Load the csv file as a pyspark dataframe
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df.show(5)



+----------+------------+--------+------------------+----------+--------------------+------------------+------------------+--------------------+--------------------+-------------------+-------------------------+--------------------+--------------------+---------------------+---------------------------+--------------+-----+-------+-----------+-------------+-----------+------------+-------------+---------------+-----------+---------+--------+----+--------+--------------------+-----------+-----+------------+-------------+----------------+------------+---------------+------------+--------------+--------------+----------------+----------------+---------------+---------------+---------------+----------------+---------------------+-----------------+------------+-----------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+-------------------+------------------------------+-----------------+--------------------+---------+
Listing ID|Last Scraped| Host ID| Host Name|Host Since| Host Location|Host Response Time|Host Response Rate|Host Acceptance Rate| Host Neighborhood|Host Listings Count|Host Total Listings Count| Street| Neighborhood|Neighborhood Cleansed|Neighborhood Group Cleansed| City|State|Zipcode| Market| Country| Latitude| Longitude|Property Type| Room Type|Accomodates|Bathrooms|Bedrooms|Beds|Bed Type| Amenities|Square Feet|Price|Weekly Price|Monthly Price|Security Deposit|Cleaning Fee|Guests Included|Extra People|Minimum Nights|Maximum Nights|Calendar Updated|Has Availability|Availability 30|Availability 60|Availability 90|Availability 365|Calendar last scraped|Number of Reviews|First Review|Last Review|Review Scores Rating|Review Scores Accuracy|Review Scores Cleanliness|Review Scores Checkin|Review Scores Communication|Review Scores Location|Review Scores Value|Cancellation Policy|Calculated host listings count|Reviews per month| Geolocation|Sentiment|
+----------+------------+--------+------------------+----------+--------------------+------------------+------------------+--------------------+--------------------+-------------------+-------------------------+--------------------+--------------------+---------------------+---------------------------+--------------+-----+-------+-----------+-------------+-----------+------------+-------------+---------------+-----------+---------+--------+----+--------+--------------------+-----------+-----+------------+-------------+----------------+------------+---------------+------------+--------------+--------------+----------------+----------------+---------------+---------------+---------------+----------------+---------------------+-----------------+------------+-----------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+-------------------+------------------------------+-----------------+--------------------+---------+
 4917301| 5/2/17|24035721| Michael| 11/22/14|Los Angeles, Cali...| null| null| null|Woodland Hills/Wa...| 1| 1|Woodland Hills/Wa...|Woodland Hills/Wa...| Woodland Hills| null| Los Angeles| CA| 91364|Los Angeles|United States|34.14775649|-118.5913378| Apartment| Private room| 2| 1.0| 1| 1|Real Bed|TV,Wireless Inter...| null| 150| null| null| null| null| 1| 0| 1| 1125| 29 months ago| null| 30| 60| 90| 365| 5/2/17| 0| null| null| null| null| null| null| null| null| null| flexible| 1| null|34.1477564923,-11...| null|
 8384432| 5/3/17| 506809| Ky| 4/13/11|Los Angeles, Cali...|within a few hours| 100| null|Woodland Hills/Wa...| 2| 2|Woodland Hills/Wa...|Woodland Hills/Wa...| Woodland Hills| null|Woodland Hills| CA| 91364|Los Angeles|United States|34.14859443|-118.5966837| Other|Entire home/apt| 2| 1.5| 1| 1|Real Bed|TV,Cable TV,Wirel...| null| 175| null| null| 500| 100| 1| 0| 2| 5| 2 weeks ago| null| 12| 42| 72| 72| 5/3/17| 2| 5/3/16| 9/20/16| 100| 10

##Create a temporary view of the dataframe 'df'

In [0]:
# Create a view or table
temp_table_name = "airbnb_sample_csv"
df.createOrReplaceTempView(temp_table_name)

In [0]:
if PYSPARK_CLI:
    csv = spark.read.csv('airbnb_sample.csv', inferSchema=True, header=True)
else:
    csv = spark.sql("SELECT * FROM airbnb_sample_csv")

csv.show(5)

+----------+------------+--------+------------------+----------+--------------------+------------------+------------------+--------------------+--------------------+-------------------+-------------------------+--------------------+--------------------+---------------------+---------------------------+--------------+-----+-------+-----------+-------------+-----------+------------+-------------+---------------+-----------+---------+--------+----+--------+--------------------+-----------+-----+------------+-------------+----------------+------------+---------------+------------+--------------+--------------+----------------+----------------+---------------+---------------+---------------+----------------+---------------------+-----------------+------------+-----------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+-------------------+------------------------------+-----------------+--------------------+---------+
Listing ID|Last Scraped| Host ID| Host Name|Host Since| Host Location|Host Response Time|Host Response Rate|Host Acceptance Rate| Host Neighborhood|Host Listings Count|Host Total Listings Count| Street| Neighborhood|Neighborhood Cleansed|Neighborhood Group Cleansed| City|State|Zipcode| Market| Country| Latitude| Longitude|Property Type| Room Type|Accomodates|Bathrooms|Bedrooms|Beds|Bed Type| Amenities|Square Feet|Price|Weekly Price|Monthly Price|Security Deposit|Cleaning Fee|Guests Included|Extra People|Minimum Nights|Maximum Nights|Calendar Updated|Has Availability|Availability 30|Availability 60|Availability 90|Availability 365|Calendar last scraped|Number of Reviews|First Review|Last Review|Review Scores Rating|Review Scores Accuracy|Review Scores Cleanliness|Review Scores Checkin|Review Scores Communication|Review Scores Location|Review Scores Value|Cancellation Policy|Calculated host listings count|Reviews per month| Geolocation|Sentiment|
+----------+------------+--------+------------------+----------+--------------------+------------------+------------------+--------------------+--------------------+-------------------+-------------------------+--------------------+--------------------+---------------------+---------------------------+--------------+-----+-------+-----------+-------------+-----------+------------+-------------+---------------+-----------+---------+--------+----+--------+--------------------+-----------+-----+------------+-------------+----------------+------------+---------------+------------+--------------+--------------+----------------+----------------+---------------+---------------+---------------+----------------+---------------------+-----------------+------------+-----------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+-------------------+------------------------------+-----------------+--------------------+---------+
 4917301| 5/2/17|24035721| Michael| 11/22/14|Los Angeles, Cali...| null| null| null|Woodland Hills/Wa...| 1| 1|Woodland Hills/Wa...|Woodland Hills/Wa...| Woodland Hills| null| Los Angeles| CA| 91364|Los Angeles|United States|34.14775649|-118.5913378| Apartment| Private room| 2| 1.0| 1| 1|Real Bed|TV,Wireless Inter...| null| 150| null| null| null| null| 1| 0| 1| 1125| 29 months ago| null| 30| 60| 90| 365| 5/2/17| 0| null| null| null| null| null| null| null| null| null| flexible| 1| null|34.1477564923,-11...| null|
 8384432| 5/3/17| 506809| Ky| 4/13/11|Los Angeles, Cali...|within a few hours| 100| null|Woodland Hills/Wa...| 2| 2|Woodland Hills/Wa...|Woodland Hills/Wa...| Woodland Hills| null|Woodland Hills| CA| 91364|Los Angeles|United States|34.14859443|-118.5966837| Other|Entire home/apt| 2| 1.5| 1| 1|Real Bed|TV,Cable TV,Wirel...| null| 175| null| null| 500| 100| 1| 0| 2| 5| 2 weeks ago| null| 12| 42| 72| 72| 5/3/17| 2| 5/3/16| 9/20/16| 100| 10

##Selecting features
In the following step, we are selecting the features that are useful for Price Prediction.

In [0]:
# Select features and label
data=csv.select("Host Listings Count",	"Host Total Listings Count",	"Neighborhood",	"Latitude","Longitude",	"Property Type",	"Room Type","Bed Type",	"Accomodates",	"Bathrooms",	"Bedrooms","Monthly Price","Cleaning Fee",	"Guests Included","Extra People","Minimum Nights","Review Scores Rating",	"Review Scores Accuracy",	"Review Scores Cleanliness",	"Review Scores Checkin","Review Scores Communication",	"Review Scores Location",	"Review Scores Value","Sentiment",col("Price").alias("label"))

data.show(5)

+-------------------+-------------------------+--------------------+-----------+------------+-------------+---------------+--------+-----------+---------+--------+-------------+------------+---------------+------------+--------------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+---------+-----+
Host Listings Count|Host Total Listings Count| Neighborhood| Latitude| Longitude|Property Type| Room Type|Bed Type|Accomodates|Bathrooms|Bedrooms|Monthly Price|Cleaning Fee|Guests Included|Extra People|Minimum Nights|Review Scores Rating|Review Scores Accuracy|Review Scores Cleanliness|Review Scores Checkin|Review Scores Communication|Review Scores Location|Review Scores Value|Sentiment|label|
+-------------------+-------------------------+--------------------+-----------+------------+-------------+---------------+--------+-----------+---------+--------+-------------+------------+---------------+------------+--------------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+---------+-----+
 1| 1|Woodland Hills/Wa...|34.14775649|-118.5913378| Apartment| Private room|Real Bed| 2| 1.0| 1| null| null| 1| 0| 1| null| null| null| null| null| null| null| null| 150|
 2| 2|Woodland Hills/Wa...|34.14859443|-118.5966837| Other|Entire home/apt|Real Bed| 2| 1.5| 1| null| 100| 1| 0| 2| 100| 10| 10| 10| 10| 10| 10| 2| 175|
 1| 1| null|34.15811293|-118.6234552| House| Private room|Real Bed| 2| 1.0| 1| 2100| 60| 1| 20| 1| null| null| null| null| null| null| null| null| 75|
 1| 1| West Hollywood|34.08838755|-118.3584776| House|Entire home/apt|Real Bed| 6| 1.5| 2| null| 50| 2| 10| 3| null| null| null| null| null| null| null| null| 146|
 3| 3| West Hollywood|34.08974187| -118.386142| Apartment| Private room|Real Bed| 1| 1.0| 1| 1900| null| 1| 0| 5| 100| 10| 10| 10| 10| 10| 10| 2| 94|
+-------------------+-------------------------+--------------------+-----------+------------+-------------+---------------+--------+-----------+---------+--------+-------------+------------+---------------+------------+--------------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+---------+-----+
only showing top 5 rows

## Data Cleaning
**Detecting and Removing Outliers:** We determine the **5th** percentile and **95th** percentile values of each of the features. Then filter the dataframe to contain data between these values.

In [0]:
# approxQuantile() to determine the 5th and 95th percentile values
outliers = data.stat.approxQuantile(['label',"Host Listings Count","Host Total Listings Count","Accomodates","Bathrooms","Bedrooms","Monthly Price", "Cleaning Fee","Guests Included","Extra People","Minimum Nights"],[0.05,0.95],0.0)
print(outliers)

import pyspark.sql.functions as f

# Filtering the dataframe by removing the outliers
data = data.filter((f.col('label') >= outliers[0][0]) & (f.col('label') <= outliers[0][1]))
data = data.filter((f.col('Host Listings Count') >= outliers[1][0]) & (f.col('Host Listings Count') <= outliers[1][1]))
data = data.filter((f.col('Host Total Listings Count') >= outliers[2][0]) & (f.col('Host Total Listings Count') <= outliers[2][1]))
data = data.filter((f.col('Accomodates') >= outliers[3][0]) & (f.col('Accomodates') <= outliers[3][1]))
data = data.filter((f.col('Bathrooms') >= outliers[4][0]) & (f.col('Bathrooms') <= outliers[4][1]))
data = data.filter((f.col('Bedrooms') >= outliers[5][0]) & (f.col('Bedrooms') <= outliers[5][1]))
data = data.filter((f.col('Monthly Price') >= outliers[6][0]) & (f.col('Monthly Price') <= outliers[6][1]))
data = data.filter((f.col('Cleaning Fee') >= outliers[7][0]) & (f.col('Cleaning Fee') <= outliers[7][1]))
data = data.filter((f.col('Guests Included') >= outliers[8][0]) & (f.col('Guests Included') <= outliers[8][1]))
data = data.filter((f.col('Extra People') >= outliers[9][0]) & (f.col('Extra People') <= outliers[9][1]))
data = data.filter((f.col('Minimum Nights') >= outliers[10][0]) & (f.col('Minimum Nights') <= outliers[10][1]))

[[40.0, 450.0], [1.0, 12.0], [1.0, 12.0], [1.0, 8.0], [1.0, 2.5], [0.0, 3.0], [930.0, 8000.0], [10.0, 180.0], [1.0, 4.0], [0.0, 50.0], [1.0, 7.0]]

##Data Cleaning
**Handling Missing Values:** Filling the missing values of numeric columns with **'0'** and string columns with **'NA'**

In [0]:
# Replacing missing values with '0' and 'NA' for numeric columns and string columns respectively
data_clean = data.na.fill(value=0).na.fill("NA")
data_clean.show(5)

+-------------------+-------------------------+--------------------+-----------+------------+-------------+---------------+--------+-----------+---------+--------+-------------+------------+---------------+------------+--------------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+---------+-----+
Host Listings Count|Host Total Listings Count| Neighborhood| Latitude| Longitude|Property Type| Room Type|Bed Type|Accomodates|Bathrooms|Bedrooms|Monthly Price|Cleaning Fee|Guests Included|Extra People|Minimum Nights|Review Scores Rating|Review Scores Accuracy|Review Scores Cleanliness|Review Scores Checkin|Review Scores Communication|Review Scores Location|Review Scores Value|Sentiment|label|
+-------------------+-------------------------+--------------------+-----------+------------+-------------+---------------+--------+-----------+---------+--------+-------------+------------+---------------+------------+--------------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+---------+-----+
 1| 1| NA|34.15811293|-118.6234552| House| Private room|Real Bed| 2| 1.0| 1| 2100| 60| 1| 20| 1| 0| 0| 0| 0| 0| 0| 0| 0| 75|
 1| 1| West Hollywood|34.08043377|-118.3707776| Apartment|Entire home/apt|Real Bed| 4| 1.5| 1| 2400| 40| 2| 10| 2| 95| 10| 9| 10| 10| 10| 10| 2| 135|
 1| 1|Westchester/Playa...| 33.9599956|-118.4088558| House|Entire home/apt|Real Bed| 5| 1.0| 2| 4300| 50| 2| 30| 2| 90| 9| 9| 9| 9| 9| 9| 2| 173|
 2| 2| Venice|33.98677834|-118.4478604| Apartment|Entire home/apt|Real Bed| 5| 1.0| 2| 6000| 100| 4| 0| 2| 86| 9| 8| 9| 9| 9| 8| 2| 200|
 2| 2| Venice|33.98741228| -118.451067| House| Private room|Real Bed| 2| 1.0| 1| 1990| 45| 1| 19| 2| 97| 10| 10| 10| 10| 10| 10| 2| 93|
+-------------------+-------------------------+--------------------+-----------+------------+-------------+---------------+--------+-----------+---------+--------+-------------+------------+---------------+------------+--------------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+---------+-----+
only showing top 5 rows

## Correlation
Determine the correlation of the label **'price'** with the features of the data indicating the **dependence** between the label and each of the features.

In [0]:
import six

df_Corr=data_clean.select("Host Listings Count","Host Total Listings Count","Neighborhood",	"Latitude","Longitude","Property Type","Room Type","Bed Type","Accomodates","Bathrooms","Bedrooms",	"Monthly Price","Cleaning Fee",	"Guests Included","Extra People","Minimum Nights","Review Scores Rating","Review Scores Accuracy","Review Scores Cleanliness","Review Scores Checkin","Review Scores Communication","Review Scores Location","Review Scores Value","Sentiment","label")

# Determining correlation using DataFrameStatFunctions.corr
for i in df_Corr.columns:
   if not( isinstance(df_Corr.select(i).take(1)[0][0], six.string_types)):
      print( "Correlation to PRICE for ", i, df_Corr.stat.corr('label',i))

Correlation to PRICE for Host Listings Count -0.05586293232183369
Correlation to PRICE for Host Total Listings Count -0.05586293232183368
Correlation to PRICE for Latitude -0.03751960489814599
Correlation to PRICE for Longitude 0.011848614046052816
Correlation to PRICE for Accomodates 0.4878581721405229
Correlation to PRICE for Bathrooms 0.2589397775426251
Correlation to PRICE for Bedrooms 0.4147797782481846
Correlation to PRICE for Monthly Price 0.7928753459408805
Correlation to PRICE for Cleaning Fee 0.5747157543356326
Correlation to PRICE for Guests Included 0.2586307076893174
Correlation to PRICE for Extra People 0.028670166239339483
Correlation to PRICE for Minimum Nights 0.10271357760320687
Correlation to PRICE for Review Scores Rating 0.009627173589364194
Correlation to PRICE for Review Scores Accuracy 0.009323850173762483
Correlation to PRICE for Review Scores Cleanliness 0.015827827466347286
Correlation to PRICE for Review Scores Checkin 0.004114880018197139
Correlation to PRICE for Review Scores Communication 0.003211808372731372
Correlation to PRICE for Review Scores Location 0.033491620689711614
Correlation to PRICE for Review Scores Value -0.009503605046631568
Correlation to PRICE for Sentiment -0.003426447550259903
Correlation to PRICE for label 1.0

##Convert the string type columns into indices using StringIndexer

In [0]:
# Converting the String type columns into indices 
data_clean = StringIndexer(inputCol='Property Type', outputCol='PropertyType_index').fit(data_clean).transform(data_clean)
data_clean = StringIndexer(inputCol='Room Type', outputCol='RoomType_index').fit(data_clean).transform(data_clean)
data_clean = StringIndexer(inputCol='Bed Type', outputCol='BedType_index').fit(data_clean).transform(data_clean)

data_clean.show(5)

+-------------------+-------------------------+--------------------+-----------+------------+-------------+---------------+--------+-----------+---------+--------+-------------+------------+---------------+------------+--------------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+---------+-----+------------------+--------------+-------------+
Host Listings Count|Host Total Listings Count| Neighborhood| Latitude| Longitude|Property Type| Room Type|Bed Type|Accomodates|Bathrooms|Bedrooms|Monthly Price|Cleaning Fee|Guests Included|Extra People|Minimum Nights|Review Scores Rating|Review Scores Accuracy|Review Scores Cleanliness|Review Scores Checkin|Review Scores Communication|Review Scores Location|Review Scores Value|Sentiment|label|PropertyType_index|RoomType_index|BedType_index|
+-------------------+-------------------------+--------------------+-----------+------------+-------------+---------------+--------+-----------+---------+--------+-------------+------------+---------------+------------+--------------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+---------+-----+------------------+--------------+-------------+
 1| 1| NA|34.15811293|-118.6234552| House| Private room|Real Bed| 2| 1.0| 1| 2100| 60| 1| 20| 1| 0| 0| 0| 0| 0| 0| 0| 0| 75| 1.0| 1.0| 0.0|
 1| 1| West Hollywood|34.08043377|-118.3707776| Apartment|Entire home/apt|Real Bed| 4| 1.5| 1| 2400| 40| 2| 10| 2| 95| 10| 9| 10| 10| 10| 10| 2| 135| 0.0| 0.0| 0.0|
 1| 1|Westchester/Playa...| 33.9599956|-118.4088558| House|Entire home/apt|Real Bed| 5| 1.0| 2| 4300| 50| 2| 30| 2| 90| 9| 9| 9| 9| 9| 9| 2| 173| 1.0| 0.0| 0.0|
 2| 2| Venice|33.98677834|-118.4478604| Apartment|Entire home/apt|Real Bed| 5| 1.0| 2| 6000| 100| 4| 0| 2| 86| 9| 8| 9| 9| 9| 8| 2| 200| 0.0| 0.0| 0.0|
 2| 2| Venice|33.98741228| -118.451067| House| Private room|Real Bed| 2| 1.0| 1| 1990| 45| 1| 19| 2| 97| 10| 10| 10| 10| 10| 10| 2| 93| 1.0| 1.0| 0.0|
+-------------------+-------------------------+--------------------+-----------+------------+-------------+---------------+--------+-----------+---------+--------+-------------+------------+---------------+------------+--------------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+---------+-----+------------------+--------------+-------------+
only showing top 5 rows

##Split the Data
Split the data into train data and test data. We have split the data in the ratio of **70:30**

In [0]:
# Split the data for random forest regression
splits = data_clean.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")

print ("Training Rows:", train.count(), " Testing Rows:", test.count())

Training Rows: 3426 Testing Rows: 1452

## Define the Pipeline
Define a pipeline that creates a feature vector and trains a regression model
1. A **VectorAssembler** that combines categorical features into a single vector.
2. A **Vector Indexer** that creates indices for a vector of categorical features.
3. A **VectorAssembler** that creates a vector of continuous numeric features.
4. A **MinMaxScaler** to normalize the continuous numeric features.
5. A **VectorAssembler** that creates a vector of categorical and continuous features.
6. A **Random Forest Regressor** that trains a Regression model.
7. **Process pipeline** with the series of transformations above.

In [0]:
# Combine Categorical features into a single vector
catVect = VectorAssembler(inputCols =['PropertyType_index', 'RoomType_index','BedType_index'], outputCol="catFeatures")

# Create indices for the vector of categorical features
catIdx = VectorIndexer(inputCol = catVect.getOutputCol(), outputCol = "idxCatFeatures").setHandleInvalid("skip")

#Create a vector of the numeric features
numVect = VectorAssembler(inputCols = ["Host Listings Count","Host Total Listings Count","Latitude","Longitude","Accomodates","Bathrooms","Bedrooms","Monthly Price","Cleaning Fee","Guests Included","Extra People","Minimum Nights","Review Scores Rating","Review Scores Accuracy","Review Scores Cleanliness","Review Scores Checkin","Review Scores Communication","Review Scores Location","Review Scores Value","Sentiment"], outputCol="numFeatures")

# Scale the numeric features
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normFeatures")

#Create a vector of categorical and numeric features
featVect = VectorAssembler(inputCols=["idxCatFeatures", "normFeatures"],  outputCol="features")

# Random Forest Regression model
rf = RandomForestRegressor(labelCol="label", featuresCol="features",maxBins=40)

# Process the pipeline with the transformations
pipeline = Pipeline(stages=[catVect,catIdx,numVect, minMax,featVect, rf])


### Train a Regression model using Parameter Tuning
Use the  **CrossValidator** class to evaluate each combination of parameters defined in a **ParameterGrid** against multiple folds of the data split into training and validation datasets, in order to find the best performing parameters. It is used to find the best model for the data. Here the number of folds is assigned to **5**.

In [0]:
# Defining the parameter grid
paramGrid = (ParamGridBuilder()
            .addGrid(rf.maxDepth,[2,3,20])
            .addGrid(rf.minInfoGain,[0.0, 0.7])
            .build())

# Number of folds
K = 5
cv = CrossValidator(estimator=pipeline, evaluator=RegressionEvaluator(), estimatorParamMaps=paramGrid, numFolds=K)
#tvs = TrainValidationSplit(estimator=pipeline, evaluator=RegressionEvaluator(), estimatorParamMaps=paramGrid, trainRatio=0.8)

# Train the model
model = cv.fit(train)


/databricks/spark/python/pyspark/ml/util.py:800: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

### Test the Pipeline Model
The model produced by the pipeline is a transformer that will apply all of the stages in the pipeline to a specified DataFrame and apply the trained model to generate predictions. In this case, we will transform the **test** DataFrame using the pipeline to generate label predictions.

In [0]:
# Transform the test data and generate predictions by applying the trained model
prediction = model.transform(test)
predicted = prediction.select("normFeatures", "prediction", "trueLabel")
predicted.show(5)

+--------------------+------------------+---------+
 normFeatures| prediction|trueLabel|
+--------------------+------------------+---------+
[0.0,0.0,0.237129...| 78.48552631578949| 55|
[0.0,0.0,0.603862...| 53.75| 40|
[0.0,0.0,0.239840...|179.04705882352943| 150|
[0.0,0.0,0.441492...| 67.34861111111111| 45|
[0.0,0.0,0.607985...| 205.83| 160|
[0.0,0.0,0.016428...| 191.65| 300|
[0.0,0.0,0.604689...| 173.65| 100|
[0.0,0.0,0.990793...|121.62857142857142| 85|
[0.0,0.0,0.603782...| 151.775| 150|
[0.0,0.0,0.604313...| 162.25| 120|
[0.0,0.0,0.604322...| 88.765| 100|
(20,[2,3,4,6,7,8,...| 83.95| 63|
[0.0,0.0,0.225771...| 86.9557894736842| 75|
[0.0,0.0,0.020093...|170.45384615384614| 199|
[0.0,0.0,0.440945...| 243.5| 240|
(20,[2,3,4,6,7,8,...|161.56666666666666| 150|
(20,[2,3,6,7,8],[...| 80.23333333333332| 80|
[0.0,0.0,0.020509...| 93.96666666666667| 79|
[0.0,0.0,0.610837...| 56.92708333333333| 47|
[0.0,0.0,5.534325...| 145.95| 100|
+--------------------+------------------+---------+
only showing top 20 rows

## Evaluate the model
Metrics used for evaluation are **Root Mean Square Error(RMSE)** and **Co-efficient of Determination(r2)**. RMSE is measured in the same units as the predicted and actual values - so in this case, the RMSE indicates the average dollars between predicted and actual price values. r2 indicates how close the data are to the fitted regression line. **RegressionEvaluator** class is used to determine **RMSE** and **r2**.

In [0]:
# Evaluator to determine rmse
evaluator = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(prediction)

# Evaluator to determine r2
evaluator = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(prediction)

print ("Root Mean Square Error (RMSE):", rmse)
print ("Co-efficient of Determination", r2)


Root Mean Square Error (RMSE): 38.0222686081765
Co-efficient of Determination 0.680739466344954